In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
import re
import numpy as np
import pandas as pd

torch.manual_seed(0)
COUNT = [0, 0, 0, 0, 0, 0, 0, 0, 0]

def calculate_motion_features_df(df):
    feature_list = []

    # Helper function to extract x, y, z data for a body part
    def get_data(body_part, data_type):
        return df[[f"{body_part} x_{data_type}", 
                   f"{body_part} y_{data_type}", 
                   f"{body_part} z_{data_type}"]].to_numpy()

    # Process each row of the DataFrame
    for _, row in df.iterrows():
        features = {}

        # 1. Energy of Motion: sum of squared acceleration components
        def calc_energy(body_part):
            accel = row[[f"{body_part} x_acce", f"{body_part} y_acce", f"{body_part} z_acce"]].values
            return np.sum(accel ** 2)

        features['right_hand_energy'] = calc_energy('Right Hand')
        features['left_hand_energy'] = calc_energy('Left Hand')

        # 2. Movement Intensity: RMS of acceleration components
        def calc_intensity(body_part):
            accel = row[[f"{body_part} x_acce", f"{body_part} y_acce", f"{body_part} z_acce"]].values
            return np.sqrt(np.mean(accel ** 2))

        features['right_hand_intensity'] = calc_intensity('Right Hand')
        features['left_hand_intensity'] = calc_intensity('Left Hand')

        # 3. Hand-Arm Angle: Angle between hand and forearm acceleration vectors
        def calc_segment_angle(part1, part2):
            vec1 = row[[f"{part1} x_acce", f"{part1} y_acce", f"{part1} z_acce"]].values
            vec2 = row[[f"{part2} x_acce", f"{part2} y_acce", f"{part2} z_acce"]].values
            dot_product = np.dot(vec1, vec2)
            magnitudes = np.linalg.norm(vec1) * np.linalg.norm(vec2)
            return np.arccos(np.clip(dot_product / magnitudes, -1.0, 1.0)) * 180.0 / np.pi if magnitudes != 0 else 0

        features['right_hand_arm_angle'] = calc_segment_angle('Right Hand', 'Right Forearm')
        features['left_hand_arm_angle'] = calc_segment_angle('Left Hand', 'Left Forearm')

        # 4. Wrist Rotation: Norm of cross product of hand and forearm acceleration vectors
        def calc_wrist_rotation(hand, forearm):
            hand_accel = row[[f"{hand} x_acce", f"{hand} y_acce", f"{hand} z_acce"]].values
            forearm_accel = row[[f"{forearm} x_acce", f"{forearm} y_acce", f"{forearm} z_acce"]].values
            return np.linalg.norm(np.cross(hand_accel, forearm_accel))

        features['right_wrist_rotation'] = calc_wrist_rotation('Right Hand', 'Right Forearm')
        features['left_wrist_rotation'] = calc_wrist_rotation('Left Hand', 'Left Forearm')

        # 5. Hands Symmetry: Norm of difference between left and right hand accelerations
        def calc_symmetry(part1, part2):
            accel1 = row[[f"{part1} x_acce", f"{part1} y_acce", f"{part1} z_acce"]].values
            accel2 = row[[f"{part2} x_acce", f"{part2} y_acce", f"{part2} z_acce"]].values
            return np.linalg.norm(accel1 - accel2)

        features['hands_symmetry'] = calc_symmetry('Right Hand', 'Left Hand')

        # 6. Posture Stability: Variance of accelerations across trunk sensors
        def calc_stability():
            trunk_parts = ['Pelvis', 'L5', 'L3', 'T12', 'T8']
            accels = np.array([row[[f"{part} x_acce", f"{part} y_acce", f"{part} z_acce"]].values for part in trunk_parts])
            return np.var(accels)

        features['posture_stability'] = calc_stability()

        # 7. Movement Efficiency: Ratio of direct path to actual path
        def calc_efficiency(body_part):
            accel = row[[f"{body_part} x_acce", f"{body_part} y_acce", f"{body_part} z_acce"]].values
            direct_path = np.linalg.norm(accel)
            actual_path = np.sum(np.abs(accel))
            return direct_path / actual_path if actual_path != 0 else 1

        features['right_hand_efficiency'] = calc_efficiency('Right Hand')
        features['left_hand_efficiency'] = calc_efficiency('Left Hand')

        # 8. Kinetic Power: Dot product of acceleration and velocity (proxy for power)
        def calc_kinetic_power(body_part):
            accel = row[[f"{body_part} x_acce", f"{body_part} y_acce", f"{body_part} z_acce"]].values
            velocity = row[[f"{body_part} x", f"{body_part} y", f"{body_part} z"]].values
            return np.sum(accel * velocity)

        features['right_hand_kinetic_power'] = calc_kinetic_power('Right Hand')
        features['left_hand_kinetic_power'] = calc_kinetic_power('Left Hand')

        feature_list.append(features)

    return pd.DataFrame(feature_list)

def categorize_sharpness(sharpness):
    if sharpness >= 85:
        return 0
    elif 70 <= sharpness < 85:
        return 1
    else:
        return 2    

def extract_and_categorize_sharpness(filename):
    # Extract sharpness value using regex (assumes sharpness is the number before the last dash)
    sharpness_value = int(re.search(r'-([0-9]+)-', filename).group(1))
    return categorize_sharpness(sharpness_value)

def split_to_chunk(df, frame_size=60, step=5):
    # Check and remove any unnecessary columns
    df = df.drop(columns=['Unnamed: 0', 'Frame', 'Marker', 'Frame_acce'], errors='ignore')
    # df["test"] = 0
    # df = df.drop(columns=['Frame', 'Marker', 'Frame_acce'], errors='ignore')
    # motion_features_df = calculate_motion_features_df(df)

    if 'Label' not in df.columns:
        raise ValueError("DataFrame must contain 'Label' column")

    # Split into chunks based on changes in label value
    chunks = []
    current_chunk = [df.iloc[0]]

    for i in range(1, len(df)):
        if df['Label'].iloc[i] == df['Label'].iloc[i - 1]:
            current_chunk.append(df.iloc[i])
        else:
            chunks.append(pd.DataFrame(current_chunk))
            current_chunk = [df.iloc[i]]
            # display(pd.DataFrame(current_chunk))
    chunks.append(pd.DataFrame(current_chunk))  # Append the last chunk

    # print("Total chunks:", len(chunks))

    samples, labels = [], []
    # output_dir = "chunk_output"
    
    # Iterate through each chunk and create samples
    for chunk_idx, chunk in enumerate(chunks):
        # print label of chunks
        # print("chunk label", chunk['Label'].iloc[0])
        # add to COUNT
        # COUNT[int(chunk['Label'].iloc[0])] += 1
        if len(chunk) >= frame_size:
            for start in range(0, len(chunk) - frame_size + 1, step):
                sample = chunk[start:start + frame_size]
                samples.append(sample.drop(columns=['Label']))
                labels.append(sample['Label'].iloc[0])  # Use the first label in the sample

    # print("Generated samples:", len(samples), "Generated labels:", len(labels))
    # print("labels", labels)
    return samples, labels


class ActivityDataset(Dataset):
    def __init__(self, root, scaler=None):
        self.root = root
        self.scaler = scaler
        self.data = []
        self.label = []
        
        for file in tqdm(os.listdir(self.root)):
            if file.endswith(".csv"):  # Ensure only CSV files are processed
                # print(f"Processing {file}")
                df = pd.read_csv(os.path.join(self.root, file))
                try:
                    samples, labels = split_to_chunk(df)
                    self.data.extend(samples)
                    self.label.extend(labels)
                except ValueError as e:
                    print(file)
                # labels = [extract_and_categorize_sharpness(file)] * len(samples)
                # self.label.extend(labels)
        
        # Normalize data if a scaler is provided
        if self.scaler is not None:
            self.data = [self.scaler.transform(sample) for sample in self.data]
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        # Convert DataFrame to NumPy array and then to a PyTorch tensor
        features = torch.tensor(self.data[idx].to_numpy(), dtype=torch.float32)
        label = torch.tensor(self.label[idx], dtype=torch.long)
        return features, label


In [ ]:
dataset = ActivityDataset(root="./processed_data")
len(dataset)

In [ ]:
dataset.data[0].shape

In [ ]:
import numpy as np

# Flatten each sample in the dataset
X = [sample.to_numpy().flatten() for sample in dataset.data]  # Features
y = dataset.label  # Labels

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)


In [ ]:
df = pd.DataFrame(y)

In [ ]:
df.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
y_val

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on the validation set
y_pred = model.predict(X_val)

# Evaluate performance
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Detailed classification report
print(classification_report(y_val, y_pred))


In [ ]:
# Assuming y_val and y_pred are defined
report_dict = classification_report(y_val, y_pred, output_dict=True)

# Convert the dictionary to a DataFrame
report_df = pd.DataFrame(report_dict).transpose()

# Display the DataFrame
display(report_df)

In [ ]:
import joblib

# Save the model
joblib.dump(model, 'RF_knife_processed_activity.pkl')

# Load the model (when needed)
# loaded_model = joblib.load('traditional_model.pkl')

In [ ]:
label_df = pd.DataFrame(dataset.label)

In [ ]:
label_df.value_counts()

In [ ]:
COUNT = [label_df[label_df[0] == i].count().values[0] for i in range(3)]

In [ ]:
COUNT

In [ ]:
from torch.utils.data import random_split, DataLoader
# Split sizes
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for training and validation sets
batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torch.nn.functional as F
import math


class TransformerClassifier(nn.Module):
    def __init__(
        self, 
        input_dim: int,
        hidden_dim: int,
        num_classes: int,
        num_heads: int = 8,
        num_layers: int = 3,
        dim_feedforward: int = 256,
        dropout: float = 0.1,
        max_seq_length: int = 60
    ):
        super().__init__()
        
        assert hidden_dim % num_heads == 0, f"hidden_dim ({hidden_dim}) must be divisible by num_heads ({num_heads})"
        
        self.linear = nn.Linear(input_dim, hidden_dim)
        self.layer_norm = nn.LayerNorm(hidden_dim)  # Layer norm is preferred over batch norm for transformers
        
        # Improved positional encoding with register_buffer instead of Parameter
        position = torch.arange(max_seq_length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_dim, 2) * (-math.log(10000.0) / hidden_dim))
        pe = torch.zeros(1, max_seq_length, hidden_dim)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pos_embedding', pe)
        
        # Add dropout after input embedding
        self.dropout = nn.Dropout(dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True  # Important for newer PyTorch versions
        )
        
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Add layer norm before final classification
        self.final_layer_norm = nn.LayerNorm(hidden_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x shape: [batch_size, seq_len, input_dim]
        
        # Input embedding
        x = self.linear(x)
        x = self.layer_norm(x)
        
        # Add positional encoding
        x = x + self.pos_embedding[:, :x.size(1)]
        x = self.dropout(x)
        
        # Transformer layers
        x = self.transformer(x)
        
        # Global average pooling and classification
        x = x.mean(dim=1)  # [batch_size, hidden_dim]
        x = self.final_layer_norm(x)
        x = self.fc(x)
        
        return x

In [ ]:
import torch
import math
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
from tqdm import tqdm

# Print the label distribution first
print("Label distribution:", COUNT)

# Calculate class weights with protection against zero
def calculate_class_weights(count_list):
    total_samples = sum(count_list)
    # Add a small epsilon to avoid division by zero
    weights = [total_samples/(len(count_list) * (c + 1e-6)) if c == 0 else total_samples/(len(count_list) * c) 
              for c in count_list]
    print(weights)
    return torch.FloatTensor(weights)

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_classes=9):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            dropout=0.2,
            bidirectional=True
        )
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        features = lstm_out.mean(dim=1)
        return self.fc(features)

def train_model(
    model, 
    train_loader, 
    val_loader, 
    class_weights,
    num_epochs=50,
    lr=1e-3,
    weight_decay=0.01,
    patience=7,
    log_dir='./runs'
):
    # If log_dir not found
    if not os.path.exists(log_dir):
        os.makedirs(log_dir, exist_ok=True)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)
    
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        epoch_preds = []
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for features, labels in progress_bar:
            features, labels = features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            _, preds = torch.max(outputs, dim=1)
            epoch_preds.extend(preds.cpu().numpy())
            
            train_loss += loss.item()
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)
            
            progress_bar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'acc': f"{train_correct/train_total:.4f}",
                'preds': f"{preds[:8]}",
            })
        
        # Monitor prediction distribution
        pred_dist = np.bincount(epoch_preds, minlength=len(COUNT))
        # print("\nPrediction distribution:", pred_dist)
        # print("Actual distribution:", COUNT)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_preds = []
        
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)
                
                _, preds = torch.max(outputs, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_loss += loss.item()
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
        
        val_acc = val_correct / val_total
        scheduler.step(val_acc)
        
        # Print detailed metrics
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {train_correct/train_total:.4f}")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc:.4f}")
        print("Validation prediction distribution:", np.bincount(val_preds, minlength=len(COUNT)))
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), f"{log_dir}/best_model.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break

        torch.save(model.state_dict(), f"{log_dir}/last_model.pt")

In [ ]:
# Initialize the model and start training
class_weights = calculate_class_weights(COUNT)
print("\nClass weights:", class_weights)

first_features, _ = dataset[0]
input_dim = first_features.shape[1]

model = LSTMClassifier(
    input_dim=input_dim,
    hidden_dim=64,
    num_classes=len(COUNT)
)

model = TransformerClassifier(
    input_dim=152,          # feature dimension
    hidden_dim=256,         # embedding size
    num_classes=len(COUNT),
    num_heads=8,            # number of heads
    num_layers=3,
    dim_feedforward=512,    # feed-forward dimension
    dropout=0.1,
    max_seq_length=60      # sequence length
)

# model.load_state_dict(torch.load('/kaggle/working/runs/simple_model/best_model.pt'))

config = {
    'num_epochs': 100,
    'learning_rate': 1e-4,
    'weight_decay': 0.01,
    'patience': 7,
    'log_dir': './runs/lstm_knife_raw'
}

train_model(
    model=model,
    train_loader=train_dataloader,
    val_loader=val_dataloader,
    class_weights=class_weights.to('cuda' if torch.cuda.is_available() else 'cpu'),
    num_epochs=config['num_epochs'],
    lr=config['learning_rate'],
    weight_decay=config['weight_decay'],
    patience=config['patience'],
    log_dir=config['log_dir']
)

In [ ]:
def evaluate_model(model, val_loader, class_weights=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()

    val_loss = 0.0
    val_correct = 0
    val_total = 0
    val_preds = []
    val_labels = []

    criterion = nn.CrossEntropyLoss(weight=class_weights) if class_weights is not None else nn.CrossEntropyLoss()

    with torch.no_grad():
        for features, labels in tqdm(val_loader, desc="Evaluating"):
            features, labels = features.to(device), labels.to(device)

            outputs = model(features)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    print(f"\nValidation Loss: {val_loss / len(val_loader):.4f}")
    print(f"Validation Accuracy: {val_acc:.4f}")
    print("Validation Prediction Distribution:", np.bincount(val_preds, minlength=len(COUNT)))
    print("Validation True Label Distribution:", np.bincount(val_labels, minlength=len(COUNT)))

    return val_acc, val_preds, val_labels

model = TransformerClassifier(
    input_dim=152,          # Your feature dimension
    hidden_dim=256,         # Increased embedding size
    num_classes=len(COUNT),
    num_heads=8,            # Increased number of heads
    num_layers=3,
    dim_feedforward=512,    # Increased feed-forward dimension
    dropout=0.1,
    max_seq_length=60      # Your sequence length
)

# model = SimpleActivityClassifier(
#     input_dim=input_dim,
#     hidden_dim=64,
#     num_classes=len(COUNT)
# )

state_dict = torch.load('./weights/trans_256_512.pt')
model.load_state_dict(state_dict)
model.eval()
# Evaluate the model on the validation set
val_acc, val_preds, val_labels = evaluate_model(model, val_dataloader)

In [ ]:
import pandas as pd

# Calculate accuracy per class
class_accuracies = {}
for class_label in set(val_labels):
    class_indices = [i for i, label in enumerate(val_labels) if label == class_label]
    class_correct = sum([1 for i in class_indices if val_preds[i] == val_labels[i]])
    class_accuracies[class_label] = class_correct / len(class_indices)

# Convert to DataFrame
accuracy_df = pd.DataFrame(list(class_accuracies.items()), columns=['Class', 'Accuracy'])

# Display the DataFrame
display(accuracy_df)

from sklearn.metrics import precision_score, recall_score, classification_report

# Calculate precision, recall, and support for each class
precision_per_class = {}
recall_per_class = {}
support_per_class = {}

for class_label in set(val_labels):
    class_indices = [i for i, label in enumerate(val_labels) if label == class_label]
    true_positives = sum([1 for i in class_indices if val_preds[i] == val_labels[i]])
    predicted_positives = sum([1 for i, pred in enumerate(val_preds) if pred == class_label])
    actual_positives = len(class_indices)
    
    precision = true_positives / predicted_positives if predicted_positives > 0 else 0.0
    recall = true_positives / actual_positives if actual_positives > 0 else 0.0
    
    precision_per_class[class_label] = precision
    recall_per_class[class_label] = recall
    support_per_class[class_label] = actual_positives

# Convert to DataFrame
metrics_df = pd.DataFrame({
    'Class': precision_per_class.keys(),
    'Accuracy': [class_accuracies[c] for c in precision_per_class.keys()],
    'Precision': precision_per_class.values(),
    'Recall': recall_per_class.values(),
    'Support': support_per_class.values(),
})

# Display the DataFrame
display(metrics_df)

# Alternatively, use sklearn classification_report for detailed metrics
report = classification_report(val_labels, val_preds, target_names=[str(cls) for cls in set(val_labels)], output_dict=True)
report_df = pd.DataFrame(report).transpose()
display(report_df)


In [ ]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score

def permutation_importance(model, dataloader, device, feature_idx):
    """
    Calculate the permutation importance of a feature.
    :param model: Trained PyTorch model
    :param dataloader: DataLoader for evaluation data
    :param device: Device to run the model on ('cpu' or 'cuda')
    :param feature_idx: Index of the feature to permute
    :return: Drop in accuracy after permutation
    """
    model.eval()
    original_preds, permuted_preds, true_labels = [], [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Original predictions
            outputs = model(inputs)
            _, preds = torch.max(outputs, dim=1)
            original_preds.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            
            # Permute the feature
            permuted_inputs = inputs.clone()
            permuted_inputs[:, :, feature_idx] = permuted_inputs[:, :, feature_idx][torch.randperm(inputs.size(0))]
            
            # Predictions after permutation
            permuted_outputs = model(permuted_inputs)
            _, permuted_preds_batch = torch.max(permuted_outputs, dim=1)
            permuted_preds.extend(permuted_preds_batch.cpu().numpy())

    # Calculate accuracy
    original_acc = accuracy_score(true_labels, original_preds)
    permuted_acc = accuracy_score(true_labels, permuted_preds)

    importance = original_acc - permuted_acc
    print(f"Original Accuracy: {original_acc:.4f}, Permuted Accuracy: {permuted_acc:.4f}, Importance: {importance:.4f}")
    return importance




In [ ]:
# Example usage with the permutation importance method
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
feature_idx = 20  # Index of the new feature
importance = permutation_importance(model, val_dataloader, device, feature_idx)

In [ ]:
# Set the model to evaluation mode
model.eval()

# Get some samples from the validation dataloader
test_samples, test_labels = next(iter(val_dataloader))

# Move the samples to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_samples, test_labels = test_samples.to(device), test_labels.to(device)

# Perform inference
with torch.no_grad():
    outputs = model(test_samples)
    _, preds = torch.max(outputs, dim=1)

# Print the predictions and the actual labels
print("Predictions:", preds.cpu().numpy())
print("Actual labels:", test_labels.cpu().numpy())